In [1]:
#Input layer - CNN / Dense Layer/ Convoluted Layer 
# i. ReLU layer
# ii. Pooling layer
#   i. batch Normalization
#   ii. Dropout layer
#   iii. Upsampling

# 2. Hidden layer - NN
# 3. Output layer - FCNN

In [2]:
# 1. Tensor Flow
# 2. Pytorch
# 3. MaxNet

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [8]:
# this sectionm is for model
import tensorflow as tf

# this section for algebraic analysis
import numpy as np

# This section for vizualization
import matplotlib.pyplot as plt


import warnings
warning.filterwarnings('ignore')
%matplotlib inline

NameError: name 'warning' is not defined

In [10]:
class NeuralNetwork:
    
    def _init_(self,layers):
          self.layers=layers
          self.L=len(layers)  
            
          self.number_feature=layers[0]
          self.number_class=layers[-1]
          
        # creating weight an dbias
          self.W={}
          self.b={}
    
          self.dw={}
          self.db={}
    
          self.setup()
          
    def setup(self):
        for i in range(1,self.L):
            self.W[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],self.layers[i-1])))
            self.b[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],1)))

In [ ]:
# 1. dtype="float32"
# 2. dtype=np.float32
# 3. dtype=tf.float32
# 4. dtype=torch.float32

In [11]:
class NeuralNetwork(NeuralNetwork):
    
    def forwardpass(self,A):
        A=tf.convert_to_tensor(A,dtype=float32)
        for i in range(1,self.L):
            Z=tf.matmul(A,tf.transpose(self.W[i]))+tf.transpose(self.b[i]) 
            if i!=self.L-1:
                A=tf.nn.relu(Z)
            else:
                A=Z
        return A            

In [12]:
# We will us ethis section for computing loss function and Upgrading the previous parameters
class NeuralNetwork(NeuralNetwork):
    def compute_loss(self, A,Y):  
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(Y,A))
    
    def upgrade_parameters():   
      for j in range(1,self.L):
        self.W[j].assing_sub(lr*self.dw[j])
        self.b[j].assing_sub(lr*self.db[j])

In [13]:
class NeuralNetwork(NeuralNetwork):
   def predict(self,x):
    A=self.forwardPass(x)
    return tf.argmax(tf.nn.softmax(A),axis=1)

   def info(self):
    num_params=0
    for i in range(1,self.L):
        num_params+=self.W[i].shape[0]*self.W[i].shape[1]
        num_params+=self.b[i].shape[0]
            
        print("Number of Feature:{}".format(self.number_feature))
        print("Total number of class is:{}".format(self.number_class))
        
        print("Hidden Layer information is:{}".format())
        for j in range(1,self.L-1):
            print("layer:{},Units {}".format(j,self.layers[j]))
        print("Total Number of parameters: {}".format(num_params))    

In [ ]:
#Training Start

# class NeuralNetwork(NeuralNetwork):
#     def train(self,x_train, y_train, x_test, y_test, epochs, step_per_epochs, batch_size,lr):
#         history={"val_loss":[],
#                  "train_loss":[],
#                 "val_acc":[]}

In [14]:
np.argmax([0.12,0.09])

0

In [ ]:
#Training Start

class NeuralNetwork(NeuralNetwork):
    def train(self,x_train, y_train, x_test, y_test, epochs, step_per_epochs, batch_size,lr):
        history={"val_loss":[],
                 "train_loss":[],
                "val_acc":[]}
        
        for e in range(0,epochs)
            training_loss_epochs=0.0
            print("Epochs {}".format(e),end="|")
            for i in range(step_per_epochs):
                x_batch=x_train[i*batch_size:(i+1)*batch_szie]
                y_batch=y_train[i*batch_size:(i+1)*batch_szie]
                batch_loss=self.training_loss_on_batch(x_batch,y_batch,lr)
                epochs_loss_train+=batch_loss
                
                if i %int(step_per_epochs/10)==0:
                    print(end=".")
                    
            history['train_loss'].append(epochs_loss_train/step_per_epochs) 
         
        valA=self.forwardPass(x_test)
        history['val_acc'].append(self.compute_loss(y_test, valA).numpy())
        
        valuePr=self.predict(x_test)
        history['val_acc'].append(np.mean(np.argmax(y_test,axis=1)==valuePr.numpy()))
        print("Value Accuracy")